# Import Required Libraries

In [40]:
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions

from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import load_img, img_to_array

from keras.models import Model

from keras.layers import Flatten, Dense

from keras.optimizers import Adam

# Model MK1  ( Checking Pre-Treined VGG16)

In [41]:
model = VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=(224, 224, 3), pooling=None)
# weights='imagenet'
# input_shape=(224,224,3)
# classes=4

In [42]:
img = load_img('/home/rodrigo/Batch/159/pngs/CT11.png')
# convert the image pixels to a numpy array
image = img_to_array(img)
print(image.shape)
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
print(image.shape)

(224, 224, 3)
(1, 224, 224, 3)


In [44]:
# prepare the image for the VGG model
image = preprocess_input(image)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [45]:
# predict the probability across all output classes
yhat = model.predict(image)

In [47]:
# convert the probabilities to class labels
label = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = label[0][0]
# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))

gong (16.88%)


# Model MK2 ( Remove Classification Layers from VGG16)

In [48]:
#add our own fully connected layers for the final classification
base_model = VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=(224, 224, 3), pooling=None)

In [50]:
x = base_model.output
#flattentit
x = Flatten()(x)
# let's add a fully-connected layer
x = Dense(4096, activation='relu')(x)
#another fully-connected layer
x = Dense(4096, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(1, kernel_initializer='normal', activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
#now we can start to fine-tune the model
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False
    
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Save Merged Label File

In [1]:
# path = '/home/rodrigo/Projects/TCC/'
# file = 'labels2.csv'
# labels = np.loadtxt(open(path + file, "rb"), delimiter=",", skiprows=1)
# exams = labels[:, 0]
# ich = [ int(x/2) for x in np.sum(labels[:,1:4], axis=1) ]
# iph = [ int(x/2) for x in np.sum(labels[:,4:7], axis=1)]
# ivh = [ int(x/2) for x in np.sum(labels[:,7:10], axis=1)]
# sdh = [ int(x/2) for x in np.sum(labels[:,10:13], axis=1)]
# edh = [ int(x/2) for x in np.sum(labels[:,13:16], axis=1)]
# sah = [ int(x/2) for x in np.sum(labels[:,16:19], axis=1)]
# result = np.transpose(np.matrix([exams, ich, iph, ivh, sdh, edh, sah]))
# np.savetxt(path + 'labels-merged.csv', result, delimiter=",")